# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
total_quantity=data.groupby(by=['CustomerID','ProductName']).agg({'Quantity':'sum'}).reset_index()

total_quantity

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot = pd.pivot_table(total_quantity,index=total_quantity.ProductName, columns=total_quantity.CustomerID,values='Quantity').fillna(0)

pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
spatial = pd.DataFrame((1/(1+squareform(pdist(pivot.T, 'euclidean')))),columns=pivot.columns, index=pivot.columns)

spatial

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
customerx=list(spatial[33].sort_values(ascending=False).index)[1:6]

customerx

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [12]:
similar_customers=total_quantity[total_quantity['CustomerID'].isin(customerx)]

similar_customers

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [21]:
agg_customers = similar_customers.groupby(by=['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False).reset_index()

agg_customers

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3
...,...,...
213,Hinge W Undercut,1
214,Ice Cream Bar - Hageen Daz To,1
215,Jagermeister,1
216,Jolt Cola - Electric Blue,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [40]:
bought_customerx=data[data['CustomerID']==33]

product_list=bought_customerx['ProductName'].to_list()

product_list

recommended=agg_customers[~agg_customers['ProductName'].isin(product_list)].head()

recommended

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3
6,Chicken - Soup Base,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [50]:
all_rec=dict()
cust_id=list(set(data['CustomerID']))

for i in cust_id:
    customerx=list(spatial[i].sort_values(ascending=False).index)[1:6]
    similar_customers=total_quantity[total_quantity['CustomerID'].isin(customerx)]
    agg_customers = similar_customers.groupby(by=['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False).reset_index()
    bought_customerx=data[data['CustomerID']==i]
    product_list=bought_customerx['ProductName'].to_list()
    recommended=list(agg_customers['ProductName'][~agg_customers['ProductName'].isin(product_list)].head())
    all_rec[i]=recommended

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [51]:
rec_df=pd.DataFrame(all_rec)

rec_df

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Cocoa Butter,"Shrimp - Baby, Warm Water",Cup - Translucent 7 Oz Clear,"Pasta - Penne, Rigate, Dry",Soupfoamcont12oz 112con,Bacardi Breezer - Tropical,Spinach - Baby,Blueberries,Ecolab - Lime - A - Way 4/4 L,Cookies - Assorted,...,V8 - Berry Blend,Ice Cream Bar - Oreo Cone,Sea Bass - Whole,Cod - Black Whole Fillet,Olive - Spread Tapenade,Cheese - Victor Et Berthold,Grouper - Fresh,Soup - Campbells Tomato Ravioli,Cake - Mini Cheesecake,Sauce - Rosee
1,Veal - Slab Bacon,Knife Plastic - White,"Veal - Brisket, Provimi,bnls",Cookies - Assorted,Anchovy Paste - 56 G Tube,Cheese - Mozzarella,Pork - Hock And Feet Attached,Mussels - Cultivated,Bread Crumbs - Panko,Table Cloth 54x72 White,...,"Pepper - Paprika, Hungarian",Soupcontfoam16oz 116con,Spinach - Baby,Thermometer Digital,Pants Custom Dry Clean,Cake - Cake Sheet Macaroon,Sherry - Dry,"Pasta - Detalini, White, Fresh",Ecolab - Solid Fusion,Scallop - St. Jaques
2,Bandage - Fexible 1x3,Towels - Paper / Kraft,Snapple Lemon Tea,Puree - Mocha,Sprouts - Baby Pea Tendrils,Cheese Cloth No 100,Vinegar - Tarragon,Vaccum Bag 10x13,Guinea Fowl,Juice - V8 Splash,...,Soupcontfoam16oz 116con,Vol Au Vents,Ice Cream Bar - Oreo Cone,"Pork - Back, Short Cut, Boneless",Sword Pick Asst,Sea Bass - Whole,"Chestnuts - Whole,canned","Cheese - Boursin, Garlic / Herbs",Wine - Redchard Merritt,"Chestnuts - Whole,canned"
3,Ezy Change Mophandle,Nantucket - Pomegranate Pear,Spinach - Baby,Cheese - Parmesan Grated,"Lemonade - Natural, 591 Ml",Bread - Italian Roll With Herbs,Bread Foccacia Whole,Turkey - Oven Roast Breast,Sunflower Seed Raw,Wine - Redchard Merritt,...,Brandy - Bar,Ketchup - Tomato,Beets - Mini Golden,Beans - Wax,Arizona - Green Tea,Squid U5 - Thailand,Napkin White - Starched,Beans - Kidney White,Mussels - Frozen,"Bread - Roll, Soft White Round"
4,Chicken - Wieners,Snapple - Iced Tea Peach,Pop Shoppe Cream Soda,Cheese - Victor Et Berthold,Sausage - Breakfast,Bread - Hot Dog Buns,Lettuce - Spring Mix,Bread - Calabrese Baguette,Bread - English Muffin,Puree - Passion Fruit,...,Vinegar - Tarragon,Anchovy Paste - 56 G Tube,Wine - Redchard Merritt,Sponge Cake Mix - Chocolate,Pail For Lid 1537,Guinea Fowl,Sprouts - Alfalfa,Wine - Blue Nun Qualitatswein,Cheese - Bocconcini,Chicken - Wieners


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.